In [1]:

from sklearn.datasets import load_wine
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import xgboost as xgb
from scipy.spatial import ConvexHull, Delaunay
import pickle
import os
import time





In [5]:
def calculate_concave_hull(coordinates, length_threshold=50):
    """
    Calculate the concave hull of the given coordinates.

    Parameters:
    coordinates (np.ndarray): Array of shape (n_samples, n_features) containing the coordinates.
    length_threshold (float): Length threshold for the concave hull algorithm.

    Returns:
    np.ndarray: The concave hull points.
    """
    hull_vertices = ConvexHull(coordinates)
    # rreturn the points
    # return area
    return coordinates[hull_vertices.vertices], hull_vertices.volume


def process_coordinates(spatial_coords, embedding_coords, k=100, length_threshold=50):
    """
    Process spatial and embedding coordinates to calculate mean distances and concave hulls.

    Parameters:
    spatial_coords (np.ndarray): Array of shape (n_samples, 2) containing the spatial coordinates.
    embedding_coords (np.ndarray): Array of shape (n_samples, 2) containing the embedding coordinates.
    k (int): Number of nearest neighbors to consider.
    threshold (float): Distance threshold to decide whether to calculate the concave hull.
    length_threshold (float): Length threshold for the concave hull algorithm.

    Returns:
    dict: Dictionary containing the results for spatial and embedding coordinates.
    """
    results = {}

    # Process spatial coordinates
    concave_hull_spatial, spatial_volume = calculate_concave_hull(
        spatial_coords, length_threshold
    )
    results["spatial"] = {
        "concave_hull": concave_hull_spatial.tolist(),
        "volume": spatial_volume,
        "density": len(spatial_coords) / spatial_volume,
        "centroid": np.mean(spatial_coords, axis=0).tolist(),
    }
    concave_hull_embedding, embedding_volume = calculate_concave_hull(
        embedding_coords, length_threshold
    )
    results["embedding"] = {
        "concave_hull": concave_hull_embedding.tolist(),
        "volume": embedding_volume,
        "density": len(embedding_coords) / embedding_volume,
        "centroid": np.mean(embedding_coords, axis=0).tolist(),
    }
    return results

def get_potential_features(csv_df):
    # Check which of the potential features are in the csv_df
    all_features = [
        "ELANE",
        "CD57",
        "CD45",
        "CD11B",
        "SMA",
        "CD16",
        "ECAD",
        "FOXP3",
        "NCAM",
        "anti_CD3",
        "anti_CD45RO",
        "Keratin_570",
        "aSMA_660",
        "CD4_488",
        "CD45_PE",
        "PD1_647",
        "CD20_488",
        "CD68_555",
        "CD8a_660",
        "CD163_488",
        "FOXP3_570",
        "PDL1_647",
        "Ecad_488",
        "Vimentin_555",
        "CDX2_647",
        "LaminABC_488",
        "Desmin_555",
        "CD31_647",
        "PCNA_488",
        "CollagenIV_647",
        "CD3",
        "CD45RO",
        "Pan-cytokeratin",
        "Aortic smooth muscle actin",
        "CD4",
        "CD45",
        "PD-1",
        "CD20",
        "CD68",
        "CD8a",
        "CD163",
        "FOXP3",
        "PD-L1",
        "E-cadherin",
        "Vimentin",
        "CDX-2",
        "Lamin-A/B/C",
        "Desmin",
        "CD31",
        "PCNA",
        "Collagen",
    ]
    all_features = list(set(all_features))
    potential_features = [
        feature for feature in all_features if feature in csv_df.columns
    ]
    return sorted(potential_features)


In [3]:
greg_df = pd.read_csv('/Users/swarchol/Research/seal/data/updated_renamed.csv')


In [ ]:
shap_values = np.load('2-5.npy')
csv_df = greg_df
def process_selection(selection_ids):
    absolute_shap_sums = np.sum(np.mean(np.abs(shap_values[selection_ids]), axis=(0)), axis=1)
    potential_features = get_potential_features(csv_df)
    feat_imp = list(zip(potential_features, absolute_shap_sums.tolist()))

    # Compute hull
    embedding_coordinates = csv_df.loc[csv_df["CellID"].isin(selection_ids)][
        ["UMAP_X", "UMAP_Y"]
    ].values

    spatial_coordinates = csv_df.loc[csv_df["CellID"].isin(selection_ids)][
        ["X_centroid", "Y_centroid"]
    ].values
    hull_results = process_coordinates(
        spatial_coordinates, embedding_coordinates
    )
    return feat_imp, hull_results

process_selection(greg_df[greg_df['kmeans'] == 7].index.tolist())

([('Aortic smooth muscle actin', 1725.393310546875),
  ('CD163', 58.762210845947266),
  ('CD20', 484.5047302246094),
  ('CD3', 451.12969970703125),
  ('CD31', 462.9090881347656),
  ('CD4', 195.77012634277344),
  ('CD45', 1331.0924072265625),
  ('CD45RO', 455.8725280761719),
  ('CD68', 126.87861633300781),
  ('CD8a', 328.91790771484375),
  ('CDX-2', 166.27279663085938),
  ('Collagen', 3023.7744140625),
  ('Desmin', 5298.03515625),
  ('E-cadherin', 760.5021362304688),
  ('FOXP3', 451.79339599609375),
  ('Lamin-A/B/C', 84.62545013427734),
  ('PCNA', 338.2778625488281),
  ('PD-1', 20.526947021484375),
  ('PD-L1', 57.668155670166016),
  ('Pan-cytokeratin', 4681.8544921875),
  ('Vimentin', 195.23403930664062)],
 {'spatial': {'concave_hull': [[14745.5, 1792.0531914893618],
    [22265.98979591837, 3846.1632653061215],
    [24357.128205128207, 13426.36752136752],
    [24452.878048780487, 13877.048780487805],
    [24405.264367816093, 15760.448275862069],
    [24400.8125, 15779.035714285714],
   

: 